In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.Lorenz96(1000) 

# Example

Run the inference algorithm

In [ ]:
config = core.Config()
config.create_time(0, 4, 40, 0, 4, 8, 0, 4, 8)

config.X_0 = np.random.random(dynamical.num_x) * 8.
config.theta = np.array([8.]) 
config.rho_2 = None
config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.array([4.5, 0.1]))
] * dynamical.num_x

config.sigma_2 = np.full(dynamical.num_x, 1.) 
config.delta = np.full(dynamical.num_x, True)
config.delta[np.random.permutation(dynamical.num_x)[:int(0.35 * dynamical.num_x)]] = False
config.gamma = np.full(dynamical.num_x, 5e-2) 
config.gamma[config.delta] = 1e-1

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = 10
config.max_iter = 1000

config.plotting_enabled = True
config.plotting_freq = 50

config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

config.debug = False

gp = core.GaussianProcessRegression(dynamical, config)
gp.run()

lpmf = core.LaplaceMeanFieldODE(dynamical, config, gp)
lpmf.run()